In [2]:
import sys
sys.path.append("/data/Beatriz/Doctorado GR/RADAR_Plataform/RADAR") # dir library

## Test with PyOD and static data

In [2]:
from RADAR.static_data.algorithms import pyod
from RADAR.static_data.static_datasets_uci import global_load
from sklearn.model_selection import train_test_split
from RADAR.static_data.preprocessing.preprocessing_static import StandardScalerPreprocessing 
from RADAR.visualization_module import DataVisualization
import numpy as np
import os

2025-10-14 21:34:51.956759: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-14 21:34:51.968953: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760470491.982943   34178 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760470491.987286   34178 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-10-14 21:34:52.004030: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

#### Define Model: example cblof

##### load data and split 

In [7]:
X,y = global_load('default_of_credit_card_clients')   #name dataset in static datasets uci repo
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,stratify=y, random_state=42)

Metadata: {'uci_id': 350, 'name': 'Default of Credit Card Clients', 'repository_url': 'https://archive.ics.uci.edu/dataset/350/default+of+credit+card+clients', 'data_url': 'https://archive.ics.uci.edu/static/public/350/data.csv', 'abstract': "This research aimed at the case of customers' default payments in Taiwan and compares the predictive accuracy of probability of default among six data mining methods.", 'area': 'Business', 'tasks': ['Classification'], 'characteristics': ['Multivariate'], 'num_instances': 30000, 'num_features': 23, 'feature_types': ['Integer', 'Real'], 'demographics': ['Sex', 'Education Level', 'Marital Status', 'Age'], 'target_col': ['Y'], 'index_col': ['ID'], 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 2009, 'last_updated': 'Fri Mar 29 2024', 'dataset_doi': '10.24432/C55S3H', 'creators': ['I-Cheng Yeh'], 'intro_paper': {'ID': 365, 'type': 'NATIVE', 'title': 'The comparisons of data mining techniques for the predictive ac

##### Define Model

In [8]:
#Preprocess Data
scaler = StandardScalerPreprocessing()
X_scaled = scaler.fit_transform(X_train)


In [9]:
kwargs = {"algorithm_": "cblof","contamination":0.1}
model_cblof = pyod.PyodAnomalyDetection(**kwargs)
print(model_cblof.get_params())

{'label_parser': None, 'algorithm_': 'CBLOF', 'alpha': 0.9, 'beta': 5, 'check_estimator': False, 'clustering_estimator': None, 'contamination': 0.1, 'n_clusters': 8, 'random_state': None, 'use_weights': False}


In [10]:
model_cblof.fit(X_scaled)


In [11]:
scores_pred = model_cblof.decision_function(X_train)* -1
print("Scores",scores_pred)

Xtest_scaled = scaler.fit_transform(X_test)
pred = model_cblof.predict(Xtest_scaled)
print("Prediction:",pred)

Scores [-436416.17619004 -149998.88779705  -23419.74930155 ... -185606.94452247
 -360001.09892637 -300006.77602229]
Prediction: [0 0 0 ... 1 0 0]


##### Anomaly Visualization

In [12]:
true = np.array(y_test).flatten()
pred = np.array(pred).astype(int)
visualizer = DataVisualization(data=X_test, plot_technique='plot_anomaly', dim_reduction_technique='PCA', y_true=true, y_pred=pred,subset_size_percent=0.2)
visualizer.fit()
visualizer.show()

##### mammographic_mass

In [17]:
X,y = global_load('mammographic_mass')   #name dataset in static datasets uci repo
mask = ~np.isnan(X).any(axis=1)
X = X[mask]
y = y[mask]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,stratify=y, random_state=42)

Metadata: {'uci_id': 161, 'name': 'Mammographic Mass', 'repository_url': 'https://archive.ics.uci.edu/dataset/161/mammographic+mass', 'data_url': 'https://archive.ics.uci.edu/static/public/161/data.csv', 'abstract': "Discrimination of benign and malignant mammographic masses based on BI-RADS attributes and the patient's age.", 'area': 'Health and Medicine', 'tasks': ['Classification'], 'characteristics': ['Multivariate'], 'num_instances': 961, 'num_features': 5, 'feature_types': ['Integer'], 'demographics': ['Age'], 'target_col': ['Severity'], 'index_col': None, 'has_missing_values': 'yes', 'missing_values_symbol': 'NaN', 'year_of_dataset_creation': 2007, 'last_updated': 'Thu Mar 28 2024', 'dataset_doi': '10.24432/C53K6Z', 'creators': ['Matthias Elter'], 'intro_paper': {'ID': 448, 'type': 'NATIVE', 'title': 'The prediction of breast cancer biopsy outcomes using two CAD approaches that both emphasize an intelligible decision process.', 'authors': 'M. Elter, R. Schulz-Wendtland, T. Witte

In [18]:
X

array([[ 5., 67.,  3.,  5.,  3.],
       [ 5., 58.,  4.,  5.,  3.],
       [ 4., 28.,  1.,  1.,  3.],
       ...,
       [ 4., 64.,  4.,  5.,  3.],
       [ 5., 66.,  4.,  5.,  3.],
       [ 4., 62.,  3.,  3.,  3.]])

In [19]:
#Preprocess Data
scaler = StandardScalerPreprocessing()
X_scaled = scaler.fit_transform(X_train)


In [20]:
kwargs = {"algorithm_": "cblof","contamination":0.1}
model_cblof = pyod.PyodAnomalyDetection(**kwargs)
print(model_cblof.get_params())

{'label_parser': None, 'algorithm_': 'CBLOF', 'alpha': 0.9, 'beta': 5, 'check_estimator': False, 'clustering_estimator': None, 'contamination': 0.1, 'n_clusters': 8, 'random_state': None, 'use_weights': False}


In [21]:
model_cblof.fit(X_scaled)


In [22]:
scores_pred = model_cblof.decision_function(X_scaled)* -1
print("Scores",scores_pred)

Xtest_scaled = scaler.fit_transform(X_test)
pred = model_cblof.predict(Xtest_scaled)
print("Prediction:",pred)

Scores [ -0.79897313  -0.89392347  -2.01819445  -0.67993001  -0.70762827
  -0.74721794  -2.90635932  -0.6627052   -0.45497605  -1.12919583
  -0.80606034  -0.53372852  -0.70163623  -0.79858276  -0.67790113
  -1.08961543  -1.24395738  -0.77035142  -1.0169058   -0.36815025
  -0.4012244   -0.64029105  -0.65629904  -0.6627052   -1.24327801
  -0.48941852  -0.76204204  -0.64238589  -1.5420875   -0.31921439
  -0.38864009  -0.84263999  -0.30987227  -0.17653905  -0.58145475
  -0.212387    -0.61864264  -2.97952132  -0.43495872  -0.52194452
  -0.76595931  -1.74068608  -0.42442138  -0.76705005  -0.70683537
  -0.47516402  -1.37906028  -0.59422149  -0.7427571   -0.70163623
  -0.54117146  -0.48049143  -0.71505195  -0.28687496  -0.85786029
  -0.52345541  -1.0564223   -0.84529527  -0.70885408  -0.31569664
  -1.13464695  -0.74721794  -0.47386072  -0.74063449  -0.6601569
  -1.21545293  -0.70606095  -0.84089606  -0.93011691  -0.86903641
  -1.15559658  -2.34335869  -1.29738229  -0.9758117   -1.60856882
  -0

#### Define model: example abod

In [ ]:
X_train,X_test,y_train,y_test=global_load('human_activity_recognition')
print("X_train:")
print(X_train.shape)
print("\nX_test:")
print(X_test.shape)
print("\ny_train:")
print(y_train.shape)
print("\ny_test:")
print(y_test.shape)


In [6]:
kwargs = {"algorithm_": "abod","contamination":0.1, "label_parser": None, "n_neighbors": 4, "method": 'fast'}
model_abod = pyod.PyodAnomalyDetection(**kwargs)
print(model_abod.get_params())

{'label_parser': None, 'algorithm_': 'ABOD', 'contamination': 0.1, 'method': 'fast', 'n_neighbors': 4}


In [9]:
model_abod.fit( X_train.astype(np.float64) )

In [10]:
scores_pred = model_abod.decision_function(X_train)* -1
print("Scores",scores_pred)


/data/Beatriz/Doctorado GR/ADL_platform/S-ADL/venv/lib/python3.10/site-packages/numpy/_core/fromnumeric.py:4008: RuntimeWarning: Degrees of freedom <= 0 for slice
  return _methods._var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/data/Beatriz/Doctorado GR/ADL_platform/S-ADL/venv/lib/python3.10/site-packages/numpy/_core/_methods.py:175: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/data/Beatriz/Doctorado GR/ADL_platform/S-ADL/venv/lib/python3.10/site-packages/numpy/_core/_methods.py:210: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Scores [4.68799730e-19 3.55438833e-15 2.81972709e-15 ... 8.32973183e-19
 2.83862659e-15 1.57303038e-13]


In [11]:
pred =  model_abod.predict(X_test)

## Pipeline

In [11]:
import numpy as np
from sklearn.model_selection import train_test_split
from RADAR.static_data.static_datasets_uci import global_load
from RADAR.static_data.preprocessing.preprocessing_static import StandardScalerPreprocessing
from RADAR.static_data.algorithms import pyod
from RADAR.metrics_module import print_metrics
from RADAR.visualization_module import DataVisualization

# 1) Data acquisition (UCI)
X, y = global_load("default_of_credit_card_clients")
y = np.ravel(y)  # ensure 1D vector

# 2) Train/test split and preprocessing (standardization)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42, stratify=y
)
scaler = StandardScalerPreprocessing()
X_train_s = scaler.fit_transform(X_train)
X_test_s  = scaler.transform(X_test)

# 3) Model selection and training (PyOD)
model = pyod.PyodAnomalyDetection(algorithm_="cblof", contamination=0.10)
model.fit(X_train_s)

# 4) Inference and metrics
y_pred = model.predict(X_test_s)
print_metrics(["Accuracy","Precision","Recall","F1","AUC_ROC"], y_test, y_pred)

# 5) Visualization (PCA + y_true vs y_pred comparison)
viz = DataVisualization(
    data=X_test_s,
    plot_technique="plot_anomaly",
    dim_reduction_technique="PCA",
    y_true=y_test,
    y_pred=y_pred,
    subset_size_percent=0.2
)
viz.fit()
viz.show()

Metadata: {'uci_id': 350, 'name': 'Default of Credit Card Clients', 'repository_url': 'https://archive.ics.uci.edu/dataset/350/default+of+credit+card+clients', 'data_url': 'https://archive.ics.uci.edu/static/public/350/data.csv', 'abstract': "This research aimed at the case of customers' default payments in Taiwan and compares the predictive accuracy of probability of default among six data mining methods.", 'area': 'Business', 'tasks': ['Classification'], 'characteristics': ['Multivariate'], 'num_instances': 30000, 'num_features': 23, 'feature_types': ['Integer', 'Real'], 'demographics': ['Sex', 'Education Level', 'Marital Status', 'Age'], 'target_col': ['Y'], 'index_col': ['ID'], 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 2009, 'last_updated': 'Fri Mar 29 2024', 'dataset_doi': '10.24432/C55S3H', 'creators': ['I-Cheng Yeh'], 'intro_paper': {'ID': 365, 'type': 'NATIVE', 'title': 'The comparisons of data mining techniques for the predictive ac